In [1]:
from dotenv import load_dotenv

from ch01_langchain_basics.ch01_02_lcel import prompt

load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
    ("placeholder", "{messages}"),
])

chain = prompt | chat

ai_msg = chain.invoke({
    "messages": [
        ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"),  # 사용자의 첫 질문
        ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."),  # 챗봇의 답변
        ("human", "방금 뭐라고 했나요?"),  # 사용자의 재확인 질문
    ]
})
print(ai_msg.content)

저축을 늘리기 위해 저축 목표를 설정하고, 매달 자동 이체를 통해 일정 금액을 저축하는 방법을 추천했습니다. 이렇게 하면 저축이 습관이 되어 더욱 효과적으로 저축을 늘릴 수 있습니다. 추가로 어떤 방법이 더 궁금하신가요?


In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

chat_history.add_user_message("방금 뭐라고 했나요?")
ai_msg = chain.invoke({"messages": chat_history.messages})
print(ai_msg.content)

저축을 늘리기 위해 저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하는 것이 좋습니다. 이렇게 하면 꾸준히 저축을 할 수 있습니다. 추가적으로, 수입의 일정 비율을 저축할 수 있도록 예산을 세우는 것도 도움이 됩니다.


In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오."),
        ("placeholder", "{chat_history}"),  # 이전 대화 이력
        ("human", "{input}"),  # 사용자의 새로운 질문
    ]
)

chat_history = ChatMessageHistory()
chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)

chain_with_message_history.invoke(
    {"input": "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    {"configurable": {"session_id": "unused"}}
).content

'저축을 늘리기 위해 몇 가지 전략을 고려해 볼 수 있습니다. 아래는 효과적인 방법들입니다:\n\n1. **예산 설정**: 수입과 지출을 철저히 분석하여 예산을 세우세요. 고정 지출과 변동 지출을 구분하고, 필요한 부분만 지출하여 여유 자금을 저축에 할당할 수 있습니다.\n\n2. **자동 저축 프로그램 활용**: 월급이 들어오자마자 정해진 금액을 자동으로 저축 계좌로 이체하도록 설정하세요. 이렇게 하면 소비를 줄이고 저축을 습관화할 수 있습니다.\n\n3. **비상 자금 마련**: 예상치 못한 지출에 대비하여 비상 자금을 마련해 두세요. 이 자금이 있으면 갑작스러운 지출을 줄이고 다른 저축 목표에 집중할 수 있습니다.\n\n4. **지출 습관 점검**: 필요하지 않은 지출 항목을 줄이거나 없애세요. 예를 들어, 자주 가는 카페, 구독 서비스 등을 재검토하여 불필요한 지출을 줄이는 것이 좋습니다.\n\n5. **더 나은 금리 찾기**: 저축 금리가 높은 계좌나 금융 상품을 찾아보세요. 이자 수익이 높을수록 저축이 더 빨리 성장합니다.\n\n6. **부가 수입 창출**: 추가적인 수입을 늘리기 위해 아르바이트, 프리랜스 일, 혹은 불필요한 물건 판매와 같은 방법을 고려해보세요.\n\n7. **지출 목표 설정**: 매달 정해진 금액을 저축하기 위해 목표를 설정하고, 해당 목표를 달성했을 때 스스로에게 작은 보상을 주세요.\n\n8. **장기 저축 목표**: 주택 구매, 자녀 교육비, 은퇴 자금 등 장기적인 저축 목표를 설정하여 그에 맞춰 저축 계획을 세우는 것이 중요합니다.\n\n이러한 방법들을 통해 저축을 늘리고 재정적 안정을 강화할 수 있습니다.'

In [5]:
chain_with_message_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content

'당신은 "저축을 늘리기 위해 무엇을 할 수 있나요?"라고 질문하셨습니다. 저축을 늘리기 위한 여러 가지 방법을 제안해드렸습니다. 추가적인 질문이나 더 궁금한 사항이 있으시면 언제든지 말씀해 주세요!'

In [7]:
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt
    | chat
)

chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 새로운 대화 내용 추가 후 체인 실행
chain_with_trimmed_history.invoke(
    {"input": "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
).content

'5년 내에 집을 구매하기 위한 재정 계획을 세우는 것은 여러 가지 요소를 고려해야 합니다. 아래의 구체적인 단계를 따라 진행해 보세요:\n\n### 1. 목표 설정\n- **구매 목표**: 어떤 유형의 주택(아파트, 단독주택 등)과 예상 가격대를 설정합니다. 지역에 따라 주택 가격이 크게 다르므로, 구체적인 시장 조사가 필요합니다.\n\n### 2. 예산 수립\n- **저축 목표**: 집을 사기 위해 필요한 보증금과 추가 비용(세금, 취득세, 보험 등)을 고려해 총 예산을 산정합니다.\n- **저축 계획**: 목표 금액을 5년(60개월)으로 나누어 매달 필요한 저축액을 계산하세요. 예를 들어, 보증금 2,000만원을 목표로 한다면 매달 약 33만원을 저축해야 합니다.\n\n### 3. 저축 및 투자 전략\n- **저축 계좌 개설**: 집 구매를 위한 저축 전용 계좌를 개설하고, 이자율이 높은 계좌를 선택하여 저축하세요.\n- **투자**: 안정적인 수익을 목표로 한다면, 저축 외에 상대적으로 위험이 적은 금융 상품(예: 적립식 펀드, 채권 등)에도 투자해 볼 수 있습니다.\n\n### 4. 신용 점검\n- **신용 점수 확인**: 대출을 받기 위해 신용 점수를 체크하고 필요하다면 개선 작업을 시작합니다. 신용 점수는 대출 조건에 큰 영향을 미치므로 중요합니다.\n- **빚 관리**: 기존의 부채가 있다면 이를 줄이거나 상환하여 신용 점수를 개선하세요.\n\n### 5. 대출 옵션 조사\n- **대출 사전승인**: 다양한 대출 상품을 비교하여 사전 승인 받는 것이 좋습니다. 이때 금리, 대출 기간 및 조건을 조사합니다.\n- **정부 지원 프로그램 활용**: 주택 구매에 대한 정부의 지원 프로그램을 확인해 보세요. 예를 들어, 저소득층을 위한 보조금이나 세금 감면 등이 있을 수 있습니다.\n\n### 6. 비상 자금 마련\n- **재정적 안전망 구축**: 예상치 못한 비용(수리비, 이사비 등)을 대비하여 비상 자금을 마련하세요. 일반적으로 3~6개월치 생활비를

In [8]:
chain_with_trimmed_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
).content


'당신은 "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"라고 질문하셨습니다. 이 질문에 대해 집 구매를 위한 재정 계획을 수립하는 방법에 대해 답변을 드렸습니다. 추가로 궁금한 점이나 다른 질문이 있으시면 언제든지 말씀해 주세요!'

In [9]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    # 대화를 요약하기 위한 프롬프트 템플릿 설정
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),  # 이전 대화 이력
            (
                "user",
                "이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오.",  # 요약 요청 메시지
            ),
        ]
    )

    # 요약 체인 생성 및 실행
    summarization_chain = summarization_prompt | chat
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})
    chat_history.clear()  # 요약 후 이전 대화 삭제
    chat_history.add_message(summary_message)  # 요약된 메시지를 대화 이력에 추가
    return True

In [10]:

# 대화 요약을 처리하는 체인 설정
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

# 요약된 대화를 기반으로 새로운 질문에 응답
print(chain_with_summarization.invoke(
    {"input": "저에게 어떤 재정적 조언을 해주셨나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content)

저는 이전 대화에서 주택 구매를 염두에 두고 5년 내에 재정 계획을 세우는 방법에 대해 조언을 드렸습니다. 구체적으로는 다음과 같은 내용이었습니다:

1. **목표 설정**: 어떤 종류의 주택을 원하고, 대략적인 예산을 설정하라는 조언을 드렸습니다. 이는 구체적인 목표를 가지고 저축하고 투자하기 위함입니다.

2. **예산 수립**: 주택 구매를 위한 총 가격과 필수 비용(세금, 보험 등)을 고려하여 매달 저축해야 할 금액을 계산하는 방법에 대해서도 언급했습니다.

3. **저축 및 투자 전략**: 주택 구매를 위한 저축 전용 계좌를 만들고, 이자율이 높은 계좌를 찾거나 위험이 적은 투자 옵션을 고려하는 것에 대해 말씀드렸습니다.

4. **신용 점검**: 대출 조건을 유리하게 만들기 위한 신용 점수 확인과 개선의 중요성에 대해 설명했습니다.

5. **대출 옵션 조사**: 다양한 대출 상품을 비교하고, 사전 승인을 받는 것의 필요성에 대해 전해드렸습니다.

6. **비상 자금 마련**: 예기치 못한 비용을 대비하여 비상 자금을 마련해두는 것이 중요하다고 강조했습니다.

7. **시장 동향 파악**: 부동산 시장의 변화를 지속적으로 모니터링하여 최적의 구매 시점을 잡는 것이 좋다고 설명했습니다.

8. **전문가 상담**: 필요할 경우, 재정 상담사와 같은 전문가의 조언을 받는 것도 좋은 방법이라고 제안했습니다.

이러한 조언을 통해 보다 체계적으로 5년 내에 원하는 주택을 구매할 수 있는 재정적 기반을 마련할 수 있도록 도와드리려 했습니다. 더 궁금한 점이나 추가적 조언이 필요한 부분이 있다면 말씀해 주세요!
